# todo: import 3 datasets, generate 3 adata

import dataset by relative path style

In [ ]:
import pandas as pd
import os

from sklearn.discriminant_analysis import StandardScaler

RELATIVE_DATA_PATH = os.path.join(os.path.pardir, 'data')

DATA_SET_NAME = 'german_socio_eco'

# Load data
data_path = os.path.join(RELATIVE_DATA_PATH, DATA_SET_NAME, f'{DATA_SET_NAME}.csv')
df_data = pd.read_csv(data_path)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_data)
# todo: write X, layers
# todo: write var - variable names
# todo: write variable types - categorical, numeric into adata.var/varm depends on the type var/varm, try being consistent with Edith's version
# todo: compute embeddings
# todo: write obsm - embeddings

array([[-1.47408022,  0.9108166 ,  0.07235934, ...,  2.23434403,
         1.42482908,  1.34651328],
       [-1.51595408,  1.21849454, -0.38148061, ...,  1.09158023,
         0.8364595 ,  0.23850783],
       [-1.20887909,  1.29541403, -0.04866465, ...,  0.36397781,
         1.22870589,  1.55613593],
       ...,
       [-0.56681321, -0.75064427, -1.16813652, ..., -0.32724449,
         0.50024832,  0.23850783],
       [-0.31557003, -0.90448324, -0.83532056, ..., -0.52198513,
         0.9765475 ,  0.68769923],
       [-0.37140185, -0.81217986, -1.16813652, ..., -0.48774502,
         2.20932185,  1.40640547]])

TODO: generate adata

In [17]:
from sklearn.discriminant_analysis import StandardScaler


def generate_adata(dataset_name):
    '''
    Data preprocessing: given dataset name, generating adata file contain versions of dataset, feature types and embeddings
    :param DATA_SET_NAME:
    :return:
    '''
    data_path = os.path.join(RELATIVE_DATA_PATH, dataset_name, f'{dataset_name}.csv')
    df_data = pd.read_csv(data_path)
    # scaler fit transform df_data
    scaler = StandardScaler()
    hd_data = scaler.fit_transform(df_data)
    adata = ad.AnnData(X=hd_data)